In [1]:
import sqlite3
import pandas as pd


In [5]:
conn = sqlite3.connect('MusicPlayer.db') 

In [6]:
sql_query = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Interactions
                               ''', conn)

In [7]:
df = pd.DataFrame(sql_query, columns = ['id', 'user_id', 'song_id' , 'like' , 'listen_count'])

In [8]:
print (df)


        id  user_id  song_id  like  listen_count
0        1        1      159     1           791
1        2        1       95     1           260
2        3        1      124     1           973
3        4        1       97     1           341
4        5        1      173     1           897
...    ...      ...      ...   ...           ...
3115  3116      156      383     1           350
3116  3117      156       62     1           359
3117  3118      156      480     1           551
3118  3119      156      112     1           947
3119  3120      156      480     1           655

[3120 rows x 5 columns]


In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import time 
import joblib
import app
from app import Interactions, db
from sqlalchemy import create_engine
import model.Recommenders as Recommenders
import model.Evaluation as Evaluation


In [15]:
sql_query = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Songs
                               ''', conn)

In [16]:
song_df = pd.DataFrame(sql_query, columns = ['id', 'name', 'path' , 'artist' , 'genre' , 'cover_photo' , 'duration' , 'total_likes' , 'total_listen_count'])

In [17]:
print(song_df)

      id                                     name  \
0      1                      Bikhoti Ka Mela.mp3   
1      2                      Chakdait Chhora.mp3   
2      3                         Jawa Pardesh.mp3   
3      4                     Kan Full Khilana.mp3   
4      5                   Latuli Teri Mukhdi.mp3   
..   ...                                      ...   
557  558  Hum Jane Pahchane Hain Kanihya Tere.mp3   
558  559   Rang De Chunirya Shyam Naam Ki (1).mp3   
559  560       Rang De Chunirya Shyam Naam Ki.mp3   
560  561        Shyam Dhani Ke Male Main Jana.mp3   
561  562        Jai Shiv Shankar Jai Mahadeva.mp3   

                                                  path         artist  \
0          Dataset\garhwali\Anil - Bikhoti Ka Mela.mp3           Anil   
1          Dataset\garhwali\Anil - Chakdait Chhora.mp3           Anil   
2             Dataset\garhwali\Anil - Jawa Pardesh.mp3           Anil   
3         Dataset\garhwali\Anil - Kan Full Khilana.mp3           Anil  

In [18]:
song_df.head()

,id,name,path,artist,genre,cover_photo,duration,total_likes,total_listen_count
0,1,Bikhoti Ka Mela.mp3,Dataset\garhwali\Anil - Bikhoti Ka Mela.mp3,Anil,garhwali,/static/images/garhwali.jpeg,7:6,6,420
1,2,Chakdait Chhora.mp3,Dataset\garhwali\Anil - Chakdait Chhora.mp3,Anil,garhwali,/static/images/garhwali.jpeg,5:39,5,426
2,3,Jawa Pardesh.mp3,Dataset\garhwali\Anil - Jawa Pardesh.mp3,Anil,garhwali,/static/images/garhwali.jpeg,6:19,7,582
3,4,Kan Full Khilana.mp3,Dataset\garhwali\Anil - Kan Full Khilana.mp3,Anil,garhwali,/static/images/garhwali.jpeg,9:44,9,27
4,5,Latuli Teri Mukhdi.mp3,Dataset\garhwali\Anil - Latuli Teri Mukhdi.mp3,Anil,garhwali,/static/images/garhwali.jpeg,7:56,5,564


In [27]:
song_df.rename(columns = {'id':'song_id'}, inplace = True)
song_df.head()

,song_id,name,path,artist,genre,cover_photo,duration,total_likes,total_listen_count
0,1,Bikhoti Ka Mela.mp3,Dataset\garhwali\Anil - Bikhoti Ka Mela.mp3,Anil,garhwali,/static/images/garhwali.jpeg,7:6,6,420
1,2,Chakdait Chhora.mp3,Dataset\garhwali\Anil - Chakdait Chhora.mp3,Anil,garhwali,/static/images/garhwali.jpeg,5:39,5,426
2,3,Jawa Pardesh.mp3,Dataset\garhwali\Anil - Jawa Pardesh.mp3,Anil,garhwali,/static/images/garhwali.jpeg,6:19,7,582
3,4,Kan Full Khilana.mp3,Dataset\garhwali\Anil - Kan Full Khilana.mp3,Anil,garhwali,/static/images/garhwali.jpeg,9:44,9,27
4,5,Latuli Teri Mukhdi.mp3,Dataset\garhwali\Anil - Latuli Teri Mukhdi.mp3,Anil,garhwali,/static/images/garhwali.jpeg,7:56,5,564


In [28]:
merged_song_df = pd.merge(df, song_df.drop_duplicates(['song_id']), on="song_id", how="left")

In [29]:
merged_song_df.head()

,id,user_id,song_id,like,listen_count,name,path,artist,genre,cover_photo,duration,total_likes,total_listen_count
0,1,1,159,1,791,Yun Sajaa Chand.mp3,Dataset\ghazal\Pankaj Udhas - Yun Sajaa Chand.mp3,Pankaj Udhas,ghazal,/static/images/ghazal.jpeg,4:53,12,40
1,2,1,95,1,260,Yeh Dil Pagal Dil Mera.mp3,Dataset\ghazal\Ghulam Ali - Yeh Dil Pagal Dil ...,Ghulam Ali,ghazal,/static/images/ghazal.jpeg,8:6,11,403
2,3,1,124,1,973,Shayad Main Zindagi Ki Sahar.mp3,Dataset\ghazal\Jagjit Singh - Shayad Main Zind...,Jagjit Singh,ghazal,/static/images/ghazal.jpeg,6:23,13,753
3,4,1,97,1,341,Haath Chute Bhi.mp3,Dataset\ghazal\Gulzar - Haath Chute Bhi.mp3,Gulzar,ghazal,/static/images/ghazal.jpeg,5:27,2,431
4,5,1,173,1,897,Khubsurat Hai Aankhen.mp3,Dataset\ghazal\Talat Aziz - Khubsurat Hai Aank...,Talat Aziz,ghazal,/static/images/ghazal.jpeg,6:39,4,897


In [30]:
merged_song_df.sort_values(by='song_id')

,id,user_id,song_id,like,listen_count,name,path,artist,genre,cover_photo,duration,total_likes,total_listen_count
741,742,38,1,1,600,Bikhoti Ka Mela.mp3,Dataset\garhwali\Anil - Bikhoti Ka Mela.mp3,Anil,garhwali,/static/images/garhwali.jpeg,7:6,6,420
3019,3020,151,1,1,420,Bikhoti Ka Mela.mp3,Dataset\garhwali\Anil - Bikhoti Ka Mela.mp3,Anil,garhwali,/static/images/garhwali.jpeg,7:6,6,420
1004,1005,51,1,1,93,Bikhoti Ka Mela.mp3,Dataset\garhwali\Anil - Bikhoti Ka Mela.mp3,Anil,garhwali,/static/images/garhwali.jpeg,7:6,6,420
1823,1824,92,1,1,715,Bikhoti Ka Mela.mp3,Dataset\garhwali\Anil - Bikhoti Ka Mela.mp3,Anil,garhwali,/static/images/garhwali.jpeg,7:6,6,420
839,840,42,1,1,441,Bikhoti Ka Mela.mp3,Dataset\garhwali\Anil - Bikhoti Ka Mela.mp3,Anil,garhwali,/static/images/garhwali.jpeg,7:6,6,420
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2723,2724,137,562,1,12,Jai Shiv Shankar Jai Mahadeva.mp3,Dataset\bhajan\Ravindra Jain - Jai Shiv Shanka...,Ravindra Jain,bhajan,/static/images/bhajan.jpeg,18:24,13,983
499,500,25,562,1,93,Jai Shiv Shankar Jai Mahadeva.mp3,Dataset\bhajan\Ravindra Jain - Jai Shiv Shanka...,Ravindra Jain,bhajan,/static/images/bhajan.jpeg,18:24,13,983
1770,1771,89,562,1,240,Jai Shiv Shankar Jai Mahadeva.mp3,Dataset\bhajan\Ravindra Jain - Jai Shiv Shanka...,Ravindra Jain,bhajan,/static/images/bhajan.jpeg,18:24,13,983
3007,3008,151,562,1,983,Jai Shiv Shankar Jai Mahadeva.mp3,Dataset\bhajan\Ravindra Jain - Jai Shiv Shanka...,Ravindra Jain,bhajan,/static/images/bhajan.jpeg,18:24,13,983
